Testing the GUI of Project (FAKE NEWS DETECTION)

In [5]:
import tkinter as tk
from tkinter import ttk, scrolledtext, messagebox
import joblib
import numpy as np
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing.sequence import pad_sequences
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import string

# Load models and vectorizer/tokenizer
try:
    logistic_model = joblib.load('logistic_model.pkl')
    naive_model = joblib.load('Naive_model.pkl')
    tfidf_vectorizer = joblib.load('tfidf_vectorizer.pkl')
    lstm_model = load_model('lstm_fake_news_model.h5')
    tokenizer = joblib.load('tokenizer.pkl')
except Exception as e:
    messagebox.showerror("Error", f"Cannot load models: {e}")
    exit()

# Preprocessing function (matches training)
def clean_text(text):
    text = str(text).lower()  
    text = text.translate(str.maketrans('', '', string.punctuation))  
    words = text.split() 
    stop_words = set(stopwords.words('english'))
    words = [WordNetLemmatizer().lemmatize(word) for word in words if word not in stop_words]  
    return ' '.join(words)  

# Prediction function
def predict_news():
    news_text = text_input.get("1.0", tk.END).strip()
    if not news_text:
        result_label.config(text="Please enter a news article.", foreground="red")
        return

    predict_button.config(state="disabled")
    result_label.config(text="Processing...", foreground="blue")
    root.update()

    try:
        cleaned_text = clean_text(news_text)
        if not cleaned_text:
            raise Exception("Text processing failed.")

        model_choice = model_var.get()
        if model_choice in ["Logistic Regression", "Naive Bayes"]:
            # Vectorize for Logistic Regression or Naive Bayes
            vectorized_text = tfidf_vectorizer.transform([cleaned_text])
            if model_choice == "Logistic Regression":
                prediction = logistic_model.predict(vectorized_text)[0]
                prob = logistic_model.predict_proba(vectorized_text)[0]
            else:
                prediction = naive_model.predict(vectorized_text)[0]
                prob = naive_model.predict_proba(vectorized_text)[0]
            result = "Real" if prediction == 1 else "Fake"
            confidence = max(prob) * 100
        else:  # LSTM
            # Tokenize and pad for LSTM
            sequence = tokenizer.texts_to_sequences([cleaned_text])
            padded_sequence = pad_sequences(sequence, maxlen=300, padding='post', truncating='post')
            prediction = lstm_model.predict(padded_sequence, verbose=0)[0][0]
            result = "Real" if prediction >= 0.5 else "Fake"
            confidence = prediction * 100 if result == "Real" else (1 - prediction) * 100

        result_label.config(
            text=f"Prediction: {result}       Confidence: {confidence:.2f}%",
            foreground="green" if result == "Real" else "red"
        )
    except Exception as e:
        messagebox.showerror("Error", f"Prediction failed: {e}")
        result_label.config(text="Prediction failed.", foreground="red")
    finally:
        predict_button.config(state="normal")

# Reset function
def reset_text():
    text_input.delete("1.0", tk.END)
    result_label.config(text="Prediction will appear here.", foreground="black")
    predict_button.config(state="normal")

# Create main window
root = tk.Tk()
root.title("Fake News Predictor")
root.geometry("600x400")
root.configure(bg="#1E3A8A")  

# Style settings
style = ttk.Style()
style.theme_use("clam")
style.configure("TButton", background="#3B82F6", foreground="white", font=("Arial", 12, "bold"), padding=10)
style.map("TButton", background=[("active", "#2563EB")])
style.configure("TLabel", background="#1E3A8A", foreground="white", font=("Arial", 12))
style.configure("TCombobox", fieldbackground="#BFDBFE", background="#3B82F6", foreground="black")

# Title
title_label = ttk.Label(root, text="Fake News Predictor", font=("Arial", 20, "bold"))
title_label.pack(pady=10)

# Model selection
model_var = tk.StringVar(value="Logistic Regression")
model_label = ttk.Label(root, text="Choose Model:")
model_label.pack()
model_menu = ttk.Combobox(root, textvariable=model_var, values=["Logistic Regression", "Naive Bayes", "LSTM"], state="readonly")
model_menu.pack(pady=5)

# Text input
input_label = ttk.Label(root, text="Enter News Article:")
input_label.pack()
text_input = scrolledtext.ScrolledText(root, height=8, width=50, bg="#BFDBFE", fg="black", font=("Arial", 11))
text_input.pack(pady=10)

# Buttons
button_frame = tk.Frame(root, bg="#1E3A8A")
button_frame.pack(pady=10)
predict_button = ttk.Button(button_frame, text="Predict", command=predict_news)
predict_button.pack(side=tk.LEFT, padx=5)
reset_button = ttk.Button(button_frame, text="Reset", command=reset_text)
reset_button.pack(side=tk.LEFT, padx=5)

# Result display
result_label = ttk.Label(root, text="Prediction will appear here.", font=("Arial", 14))
result_label.pack(pady=10)


root.mainloop()